In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time
import tqdm
import typing

_ = matplotlib.rc("text", usetex=True)
_ = matplotlib.rc("font", family="serif", serif="CMU Serif")

In [ ]:
def plot_tradeoffs(
    df: pd.DataFrame,
    horizontal_plots,
    vertical_plots,
    **kwargs,
) -> None:
    """Plot the tradeoff between cost and task performance"""
